# File Processing 
----

#### Data cleaning and formatting is done to create Pandas dataframes to be used for mapping and visualization of the data.  The primary data set comes from an api generated by CBS Sports News and hosted by Amazon Web Services.  The data set is a listing of College/University Sports Events that are scheduled to be streamed by video or audio.  The ask by CBS Sports News is to take the API that is generated weekly and create a visualization of the scheduled events to be broadcast to help anticipate staffing needs on a daily basis.  For CBS Sports News, a heat map/and or graphic visualization of the games to be broadcast by specific pub points will be used to deliver this information.   Further analysis of the events data, will be done using information gathered from a listing of Universities and Colleges to get location data to create maps and visualizations of the events held at specific locations.




In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json as js
from scipy.stats import linregress

# Import API key - usng CBS keys - not used yet
#from api_keys import sport_key
from config import gkey
from config import scorecard_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Input test file (JSON).
input_data_file01="Resources/Events.json"
input_data_file02="Resources/MERGED2018_19_PP.csv"
output_data_file = "eventsMaster.csv"


# Steps for analyzing / cleaning data

Step1 - Capture data from the CBS Sports News API, filter, clean and assemble a dataframe that has school location data and pub points data 
----

Need: 
'count'  - gives the number of events in the list
nested in 'count'
'events' - nested
 by 'contenttype'
     'eventstate' 'scheduled'
     'eventstatus' 'live'
     'eventtype' 'game'
     'is_passthrough' 'False'
     'prismid': '27a62c10-4a15-42ae-a81b-9b31c346ffb9',  [unique id]
     'schedule': {'endtimestamp': 1618181100,
                          'starttimebuffer': 0,
                          'starttimestamp': 1618163100},
     'school': 'nwst',
     'school_name': 'Northwestern State University',
     'sport': 'm-basebl',
     'sport_name': 'Baseball',



In [5]:
#Step 1
with open(input_data_file01) as f:
  data = js.load(f)
#gives a dictionary
#data.values()


In [10]:
#starttime and endtime are in epoch time 
starttime=1617768000
endtime=1618372800
#url ='https://ufp5x5qk2i.execute-api.us-east-1.amazonaws.com/prod/eventmanager/events?starttime=1617768000.001&endtime=1618372800'
url ='https://ufp5x5qk2i.execute-api.us-east-1.amazonaws.com/prod/eventmanager/events?starttime='+str(starttime)+'&endtime='+str(endtime)

#headers pulls api key from config.py, assignes it to x-api-key
headers = {"scorecard_key": scorecard_key}

#assemble request, pulls x-api-key as header to access api
req = requests.get(url, headers=headers)
print(req)

response_json = req.json()
print(js.dumps(response_json, indent=4, sort_keys=True))

<Response [403]>
{
    "message": "Forbidden"
}


In [ ]:
import pprint
#pp = pprint.PrettyPrinter(depth=4)
#pp.pprint(data)


In [ ]:
#another way to print
#pp.pprint(f'Dictionary comprehension: {data}')

In [ ]:
data.keys()

In [ ]:
#Step 1
#checking to remove passthroughs
newDict={}
print(data['count'])
newCount=0
#filter out the passthrough records
Counts=data['count']
#first look for passthrough = True
for index in range(1,Counts):
    if data['events'][index]['is_passthrough']==False:
        newDict=data
        newCount= newCount+1
    
#print(newDict)    
print(f'filtered data counts {newCount}')

In [ ]:
#Step 1
#create the master filtered data frame
ID=[]
Type=[]
Scheduled=[]
Status=[]
CType=[]
Pass=[]
Start=[]
End=[]
Event_Title=[]
School_Name=[]
School_Code=[]
Game=[]
PubPoint=[]


for index in range(1,newCount):
    try:
        ID.append(newDict['events'][index]['prismid'])
        Scheduled.append(newDict['events'][index]['eventstate'])
        Pass.append(newDict['events'][index]['is_passthrough'])
        Start.append(newDict['events'][index]['starttime'])
        End.append(newDict['events'][index]['endtime'])
        Event_Title.append(newDict['events'][index]['eventtitle'])
        School_Name.append(newDict['events'][index]['school_name'])
        School_Code.append(newDict['events'][index]['school'])
        Game.append(newDict['events'][index]['sport_name'])
        
    except ValueError:
        continue
    except KeyError:
        print(index)
        continue
             
        
  
    
event_df=pd.DataFrame(ID)
event_df['Scheduled']=Scheduled
event_df['PassThru']=Pass
event_df['Start Time']=Start
event_df['End Time']=End
event_df['Event']=Event_Title
event_df['School Name']=School_Name
event_df['School Code']=School_Code
event_df['Sport']=Game
event_df.rename(columns={0:'ID'},inplace=True)
event_df.set_index('ID',inplace=True)



In [ ]:
event_df.head()

In [ ]:
event_df.columns

In [ ]:
#Step 1
#get the pass through record values for each prismid
with open(input_data_file01) as f:
  filter_set = js.load(f)
#gives a dictionary
filter_set.values()

prismid=[]
Pass=[]
contenttype=[]
pubPoint=[]
#filter out the passthrough records
Counters=filter_set['count']

pr_count=0
content_count=0
pass_count=0
pub_count=0
#first look for passthrough = True
for index in range(1,Counts):
   try:     
    
    if filter_set['events'][index]['is_passthrough']==True:
        Pass.append(filter_set['events'][index]['is_passthrough'])
        prismid.append(filter_set['events'][index]['prismid'])
        contenttype.append(filter_set['events'][index]['contenttype'])
        pubPoint.append(filter_set['events'][index]['ingest']['primary']['pub_point'])
   except KeyError:
        continue

count_pid=len(prismid)
print(count_pid)        
#count_contenttype=len(contenttype)  
#count_pass=len(Pass)
#print(count_pass)
#print(f'pass_throughs are :{Pass}')  
count_pubs=len(pubPoint)
print(count_pubs)
#print(f'pubs are: {pubPoint}')
  
#print(f'prismids to exclude are :{prismid}')

In [ ]:
#Step 1
#create the pub poing list to merge back to the master filtered data frame
counter=0
xcount=0
prID=[]
PP=[]
# the pub points
for index in range(1,newCount):
    if newDict['events'][index]['prismid'] in prismid:
        xcount=+1
    elif newDict['events'][index]['prismid']not in prismid:
        prID.append(newDict['events'][index]['prismid'])
        PP.append(newDict['events'][index]['ingest']['primary']['pub_point'])
        
print(xcount)
print(len(prID))  
print(len(PP))

AddPP_df=pd.DataFrame(prID)
AddPP_df['PubPoint']=PP
AddPP_df.rename(columns={0:'ID'},inplace=True)
AddPP_df.set_index('ID',inplace=True)

In [ ]:
print(AddPP_df)

In [ ]:
#Step 1
#this join gets the 558 with 442 pub points
working_events=event_df.merge(AddPP_df,how='left', left_on='ID',right_on='ID')
#record checks

working_events.count()

In [ ]:
working_events.tail(25)

In [ ]:
#Step 1 - a copy of the dataframe is save to a file 
#output dataframe to CSV file - passthroughs accunted for - audio shown in pubPoint
working_events.to_csv('NewPubPoints_in_events.csv')

### Final Data Frame 
* working_events is saved NewPubPoints_in_events.csv


### XXXXXX
* Export the city data into a .csv.
* Display the DataFrame